In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')
import os
import collections

In [2]:
preDouble = "\\\\iobsdc01\\SharedDocs\\SEER_MEDICARE\\SEER_MEDICARE_STAGING"
new = os.chdir(preDouble)
print(preDouble)

\\iobsdc01\SharedDocs\SEER_MEDICARE\SEER_MEDICARE_STAGING


In [3]:
import pickle
fileObject = open('control_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
controlindices = list(b)

In [4]:
import pickle
fileObject = open('treatment_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
treatmentindices = list(b)

In [5]:
print(len(controlindices), len(treatmentindices))

1031 1031


In [6]:
dg = pd.read_pickle('goodindices_costofcare_outsaf.pickle')

Dg contains the dataframe created in costofcare.ipynb, that grabs all the rows of the outsaf claims files corresonding to controlindices and treatmentindices.

Now we need to add in the modx1 information as well as the site1 information from the pedsf files.

In [7]:
krazykols = ['patient_id','claim_from_date',
            'link_num','rec_count','hcpcs','tot_chrg','tot_line',
            'seg_line']

dg[krazykols].head(30)

,patient_id,claim_from_date,link_num,rec_count,hcpcs,tot_chrg,tot_line,seg_line
0,0244410011,12192006,1042840,1,NaN,5242.49,8,8
1,0244410011,12192006,1042840,2,NaN,5242.49,8,8
2,0244410011,12192006,1042840,3,NaN,5242.49,8,8
3,0244410011,12192006,1042840,4,NaN,5242.49,8,8
4,0244410011,12192006,1042840,5,NaN,5242.49,8,8
5,0244410011,12192006,1042840,6,66984,5242.49,8,8
6,0244410011,12192006,1042840,7,NaN,5242.49,8,8
7,0244410011,12192006,1042840,8,NaN,5242.49,8,8
8,0244410011,6242006,1285544,1,NaN,1142.80,8,8
9,0244410011,6242006,1285544,2,80048,1142.80,8,8


In [8]:
dg['claim_from_date'] = dg['claim_from_date'].apply(str)
dg['claim_from_date'] = dg['claim_from_date'].apply(lambda x: x.rjust(8, '0'))

In [9]:
dg['claim_from_date'] = pd.to_datetime(dg['claim_from_date'], format='%m%d%Y')
dg['claim_from_date_day'] = dg['claim_from_date'].dt.day
dg['claim_from_date_month'] = dg['claim_from_date'].dt.month
dg['claim_from_date_year'] = dg['claim_from_date'].dt.year

## Now need to load in the stuff from the pedsf big file.

We need to only compute the total cost of care for each patient_id, where the 'claim_from_date' is later in time than the diagnosis date.

In [10]:
df = pd.read_pickle('dfpedsfFirsts.pickle')

In [11]:
df = df.set_index('patient_id',drop=False)

In [13]:
goodindices = controlindices + treatmentindices
print(len(goodindices),len(controlindices),len(treatmentindices))

2062 1031 1031


In [14]:
df_all_good_indices = df.loc[goodindices].copy()

In [15]:
df_all_good_indices.yrdx1.head(10)

patient_id
8835051335    2009
8880063314    2008
8865006343    2007
8833080450    2010
8887079350    2006
8855021414    2011
8834065023    2008
8876047420    2010
8840048462    2011
8874062373    2008
Name: yrdx1, dtype: int64

In [16]:
df_all_good_indices.modx1.head(10)

patient_id
8835051335    Apr
8880063314    Dec
8865006343    Jul
8833080450    Jul
8887079350    Jun
8855021414    Apr
8834065023    Oct
8876047420    Jan
8840048462    Jul
8874062373    Apr
Name: modx1, dtype: object

In [18]:
df_all_good_indices.site1.value_counts()

Prostate gland               1854
Choroid                        86
Upper lobe, lung               62
Lower lobe, lung               14
Frontal lobe                   10
Cerebral meninges               6
Lower third of esophagus        6
Lung, NOS                       4
Parotid gland                   4
Parietal lobe                   4
Acoustic nerve                  2
Cervical esophagus              2
Middle third of esophagus       2
Ciliary body                    2
Main bronchus                   2
Temporal lobe                   2
Name: site1, dtype: int64